# Find the Tents

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

The problem was taken from https://www.brainbashers.com/tents.asp ; there, you can try to solve some examples manually.

## Task

Find all of the hidden tents in the forest grid.

You know that:

- Each tent is attached to one tree (so there are as many tents as there are trees).
- A tent can only be found horizontally or vertically adjacent to a tree.
- Tents are never adjacent to each other, neither vertically, horizontally, nor diagonally.
- A tree might be next to two tents but is only connected to one.

You are also given two vectors indicating how many tents are in each respective row or column of the forest grid.


## Input

You are given a positive integer $n \geq 2$, representing the size of the forest grid (assume it is a square of size $(n \times n$). You are also given vectors $\mathbf r = (r_1, \dots, r_n)$ and $\mathbf c = (c_1, \dots, c_n)$ representing the numbers of the tents in the rows and columns of the forest grid. Finally, you are given a list of coordinates of the trees $((x_1, y_1), \dots, (x_k, y_k))$.


In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 33.5 MB/s eta 0:00:00


In [ ]:
# 2x2 - Extra small (for debugging)
n = 3
r = (1, 1, 0)
c = (1, 0, 1)
trees = [(1,1), (3,2)]

In [ ]:
# 8x8 - Medium
n = 8
r = (3, 1, 1, 2, 0, 2, 0, 3)
c = (2, 1, 2, 2 ,1, 1 ,2 ,1)
trees = [(2, 1), (5, 1), (6, 1),
         (1, 2),
         (3, 3),
         (3, 4), (6, 4),
         (4, 5), (6, 5),
         (8, 7),
         (2, 8), (4, 8)]

In [ ]:
# Weekly special
n = 20
r = (7, 2, 3, 4, 3, 5, 4, 4, 4, 4, 3, 6, 3, 6, 2, 3, 6, 3, 3, 5)
c = (6, 4, 3, 5, 4, 4, 4, 3, 5, 3, 4, 3, 4, 4, 6, 3 ,4, 3, 6, 2)
trees = [(3, 1), (4, 1), (8, 1), (13, 1), (15, 1),
         (1, 2), (9, 2), (18, 2), (19, 2),
         (5, 3), (12, 3), (15, 3),
         (2, 4), (4, 4), (9, 4), (17, 4),
         (6, 5), (10, 5), (13, 5), (17, 5), (20, 5),
         (1, 6), (7, 6), (10, 6), (12, 6), (16, 6),
         (20, 7),
         (1, 8), (4, 8), (5, 8), (11, 8), (13, 8), (14, 8), (19, 8),
         (4, 9), (6, 9), (9, 9), (15, 9), (17, 9),
         (8, 10), (17, 10), (19, 10),
         (12, 11),
         (5, 12), (7, 12), (14, 12), (16, 12),
         (1, 13), (2, 13), (6, 13), (19, 13),
         (11, 14), (14, 14), (20, 14),
         (3, 15), (5, 15), (6, 15), (8, 15), (13, 15), (20, 15),
         (2, 16), (3, 16), (10, 16),
         (8, 17), (11, 17), (14, 17), (15, 17),
         (2, 18), (6, 18), (9, 18), (12, 18), (13, 18), (18, 18),
         (2, 19), (7, 19), (15, 19), (17, 19), (20, 19),
         (5, 20), (10, 20)]

## Output

You should find the coordinates $(x_i, y_i), i \in \{1,\dots,k\}$, of the individual tents.

## Model

In [ ]:
import gurobipy as g

tree_grid = [[False for _ in range(n+2)] for _ in range(n+2)]
for t_x, t_y in trees:
    tree_grid[t_y][t_x] = True

m = g.Model()

# - add variables
dirs = ["N", "E", "S", "W"]

# a[x,y,d] = 1 ~ there is a tent on coord. (x,y) linked with tree in direction dir
a = m.addVars(n+2, n+2, dirs, vtype=g.GRB.BINARY, name="a")

# - add constraints
for x in range(1, n+1):
    for y in range(1, n+1):
        # If there is not a tree, constrain the domains of neigh. fields
        if not tree_grid[y][x]:
            m.addConstr(a[x-1,y,"E"] == 0)
            m.addConstr(a[x+1,y,"W"] == 0)
            m.addConstr(a[x,y-1,"S"] == 0)
            m.addConstr(a[x,y+1,"N"] == 0)
        # if there is a tree, exactly one tent is linked to it
        else:
            m.addConstr(a[x-1,y,"E"] + a[x+1,y,"W"] + a[x,y-1,"S"] + a[x,y+1,"N"] == 1)

        # if there is a tent, there cannot be an adjacent tent
        m.addConstr(8 * (1-a.sum(x,y,"*")) >= a.sum(x+1,y,"*") + a.sum(x+1,y+1,"*") + a.sum(x+1,y-1,"*")
                                                + a.sum(x-1,y,"*") + a.sum(x-1,y+1,"*") + a.sum(x-1,y-1,"*")
                                                + a.sum(x,y+1,"*") + a.sum(x,y-1,"*"))

        # there can be only a single tent at one place
        m.addConstr(a.sum(x,y,"*") <= 1)

# columns and rows
for i in range(1, n+1):
    m.addConstr(a.sum(i, "*", "*") == c[i-1])  # c is indexed from 0
    m.addConstr(a.sum("*", i, "*") == r[i-1])  # r is indexed from 0

# no tents at trees
for t_x, t_y in trees:
    m.addConstr(a.sum(t_x,t_y,"*") == 0)

# no tents on the dummy boundaries
m.addConstr(a.sum(0, "*", "*") + a.sum(n+1,"*", "*") + a.sum("*", 0, "*") + a.sum("*", n+1, "*") == 0)

m.optimize()

tents = []
for x in range(1, n+1):
    for y in range(1, n+1):
        if a.sum(x,y,"*").getValue() > 0.5:
            tents.append((x,y))

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 377 rows, 400 columns and 3648 nonzeros
Model fingerprint: 0xe9efa230
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 377 rows and 400 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


 ##  Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(n, trees, tents, r, c):
    grid = [["." for _ in range(n+2)] for _ in range(n+2)]

    for t_x, t_y in tents:
        grid[t_y][t_x] = "X"

    for t_x, t_y in trees:
        grid[t_y][t_x] = "T"

    print("  ", end="")
    for c_cur in c:
        print(c_cur, end=" ")
    print()

    for y in range(1, n+1):
        print(r[y-1], end=" ")
        for x in range(1, n+1):
            print(grid[y][x], end=" ")

        print()

In [ ]:
visualize(n, trees, tents, r, c)

  2 1 2 2 1 1 2 1 
3 X T X . T T X . 
1 T . . . X . . . 
1 . X T . . . . . 
2 . . T X . T X . 
0 . . . T . T . . 
2 . . . X . X . . 
0 . . . . . . . T 
3 X T X T . . . X 
